In [1]:
import os, sys
os.environ['CUDA_VISIBLE_DEVICES'] = "1"
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib as plt
from PIL import Image
sys.path.append('../src')
from utils import *
from matplotlib.lines import Line2D
import matplotlib as mpl
import math
import matplotlib.image as mpimg
import scipy.stats as stats
import scipy as sp
from scipy.stats import pearsonr,binom,linregress
from ast import literal_eval
import json
from matplotlib.colors import ListedColormap
import matplotlib.patches as mpatches
from mpl_toolkits.axes_grid1 import make_axes_locatable
from scipy.stats import gaussian_kde
os.chdir("..")

In [2]:
def transpose_flat_list(flat_list, rows, cols):
    # Step 1: Convert to 2D list
    two_d_list = [flat_list[i * cols:(i + 1) * cols] for i in range(rows)]
    
    # Step 2: Transpose 2D list
    transposed = list(zip(*two_d_list))
    
    # Step 3: Flatten the list
    transposed_flat = [item for sublist in transposed for item in sublist]
    
    return transposed_flat

def make_results_figure(save_path, file_paths = [], captions = [], rows=6, columns=5):
    images = []
    for file in file_paths:
        images.append(Image.open(file))
    print(rows, columns)
    output = tileImages("", images, captions, h=rows, w=columns, use_title=2, row_captions=True, buffer=35, red_col=True)
    output.save(save_path)

In [8]:
file_paths = []
mode = "vision"
# mode = "imagery"
# stim = "simple"
# stim = "complex"
stim = "concepts"
for method in ["secondsight", "mindeye", "mindeye2" "braindiffuser", "tagaki"]:
    directory_path = f"output/mental_imagery_paper/{mode}/{method}/subject1_statistics_12.csv"
    df = pd.read_csv(directory_path)
    # df.insert(loc=1, column='Subject', value=1)
    # df = column_string_to_list(df)
    for subject in [1,2,5,7]:
        new_df = pd.read_csv(f"output/mental_imagery_paper/{mode}/{method}/subject{subject}_statistics_12.csv")
        # new_df.insert(loc=1, column='Subject', value=subject)
    #     # new_df = column_string_to_list(new_df)
        df = pd.concat([df, new_df])

    # df.head(100)
    # print(df)
    df.to_csv(f"output/mental_imagery_paper/{mode}/{method}/all_subject_statistics.csv")

    df.reset_index(drop=True, inplace=True)
    # Define the columns to be used for selecting the best row for each "ID"
    columns_to_consider = [
        # "CLIP Cosine",
        # "Brain Correlation V1",
        # "Brain Correlation V2",
        # "Brain Correlation V3",
        # "Brain Correlation V4",
        # "Brain Correlation Higher Visual",
        # "Brain Correlation NSD General",
        # "AlexNet 2 1000",
        # "AlexNet 5 1000",
        # "AlexNet 7 1000",
        # "CLIP Two-way 1000",
        # "Inception V3 1000"
    ]

    # Filter the DataFrame for ID values less than or equal to 5 and "Sample Indicator" = 12
    if stim == "simple":
        filtered_df = df[(df['ID'] <= 5) & (df['Sample Indicator'] == 12)]
    elif stim == "complex":
        filtered_df = df[(df['ID'] > 5) & (df['Sample Indicator'] == 12)]
        

    # Rank each of the relevant columns across the filtered DataFrame and sum these ranks for each row
    ranked_df = filtered_df[columns_to_consider].rank(ascending=False, method='min', na_option='keep')
    filtered_df['Rank_Sum'] = ranked_df.sum(axis=1)

    # Find the row with the lowest sum of ranks for each "ID"
    best_rows_indices_filtered = filtered_df.groupby('ID')['Rank_Sum'].idxmin()
    best_rows_filtered = filtered_df.loc[best_rows_indices_filtered].sort_values(by="ID")

    # Generate file paths for the images corresponding to the rows after filtering
    image_file_paths = [
        f"output/mental_imagery_paper/{row['Mode']}/{row['Method']}/subject{int(row['Subject'])}/{int(row['ID'])}/{int(row['Sample Count'])}.png"
        for index, row in best_rows_filtered.iterrows()
    ]

    # Generate file paths for the ground truth images corresponding to the filtered rows
    ground_truth_file_paths = [
        f"output/mental_imagery_paper/{row['Mode']}/{row['Method']}/subject{int(row['Subject'])}/{int(row['ID'])}/ground_truth.png"
        for index, row in best_rows_filtered.iterrows()
    ]
    # print(len(image_file_paths))
    if method == "secondsight":
        file_paths += ground_truth_file_paths
    file_paths += image_file_paths

captions = ["Ground Truth", "MindEye + BOI", "MindEye", "Brain Diffuser", "Tagaki et al."]
images = []
for file in file_paths:
    images.append(Image.open(file))
if mode == "vision":
    title =  f"Reconstructions for seen {stim} stimuli"
else:
    title =  f"Reconstructions for imagined {stim} stimuli"
    
    
tiled_figure = format_tiled_figure(
    images=images,
    captions=captions, 
    rows = 5, 
    cols = 6,
    red_line_index=0,  # Replace with the index of the column/row to highlight red
    buffer=20, # The buffer space in pixels between images.
    mode=3, # The mode of how captions and titles are used.
    title = title
    )
tiled_figure.save(f"output/mental_imagery_paper/image_grids/mi_{mode}_{stim}.png")
# tiled_figure.save(f"output/second_sight_paper/qualia_structure_figures/mi_{mode}_{stim}.png")

# make_results_figure(f"output/mental_imagery_paper/image_grids/mi_{mode}_{stim}.png", transpose_flat_list(file_paths, 5, 6), captions, rows=6, columns=5)
# make_results_figure(f"output/mental_imagery_paper/image_grids/mi_{mode}_{stim}_sideways.png", file_paths, captions, rows=5, columns=6)

/tmp/ipykernel_55936/3578128967.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Rank_Sum'] = ranked_df.sum(axis=1)
/tmp/ipykernel_55936/3578128967.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Rank_Sum'] = ranked_df.sum(axis=1)
/tmp/ipykernel_55936/3578128967.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [9]:
file_paths = []
# mode = "vision"
mode = "imagery"
stim = "simple"
for method in ["secondsight", "mindeye", "braindiffuser", "tagaki"]:
    
    directory_path = f"output/mental_imagery_paper/{mode}/{method}/subject1_statistics_12.csv"
    df = pd.read_csv(directory_path)
    # df.insert(loc=1, column='Subject', value=1)
    # df = column_string_to_list(df)
    for subject in [2,5,7]:
        new_df = pd.read_csv(f"output/mental_imagery_paper/{mode}/{method}/subject{subject}_statistics_12.csv")
        # new_df.insert(loc=1, column='Subject', value=subject)
    #     # new_df = column_string_to_list(new_df)
        df = pd.concat([df, new_df])

    # df.head(100)
    # print(df)
    df.to_csv(f"output/mental_imagery_paper/{mode}/{method}/all_subject_statistics.csv")

    df.reset_index(drop=True, inplace=True)
    # Define the columns to be used for selecting the best row for each "ID"
    columns_to_consider = [
        "CLIP Cosine",
        "Brain Correlation V1",
        "Brain Correlation V2",
        "Brain Correlation V3",
        "Brain Correlation V4",
        "Brain Correlation Higher Visual",
        "Brain Correlation NSD General",
        "AlexNet 2 1000",
        "AlexNet 5 1000",
        "AlexNet 7 1000",
        "CLIP Two-way 1000",
        "Inception V3 1000"
    ]

    # Filter the DataFrame for ID values less than or equal to 5 and "Sample Indicator" = 12
    filtered_df = df[(df['ID'] <= 5) & (df['Sample Indicator'] == 11)]

    # Rank each of the relevant columns across the filtered DataFrame and sum these ranks for each row
    ranked_df = filtered_df[columns_to_consider].rank(ascending=False, method='min', na_option='keep')
    filtered_df['Rank_Sum'] = ranked_df.sum(axis=1)

    # Find the row with the lowest sum of ranks for each "ID"
    best_rows_indices_filtered = filtered_df.groupby('ID')['Rank_Sum'].idxmin()
    best_rows_filtered = filtered_df.loc[best_rows_indices_filtered].sort_values(by="ID")
    
    # Generate file paths for the images corresponding to the rows after filtering
    image_file_paths = [
    f"output/mental_imagery_paper/{row['Mode']}/{row['Method']}/subject{int(row['Subject'])}/{int(row['ID'])}/low_level.png"
    for index, row in best_rows_filtered.iterrows()
    ]

    ground_truth_file_paths = [
        f"output/mental_imagery_paper/{row['Mode']}/{row['Method']}/subject{int(row['Subject'])}/{int(row['ID'])}/ground_truth.png"
        for index, row in best_rows_filtered.iterrows()
    ]
    print(len(image_file_paths), len(ground_truth_file_paths))
    if method == "mindeye":
        file_paths += ground_truth_file_paths
    file_paths += image_file_paths
    
file_paths_t = transpose_flat_list(file_paths, 4, 6)
print(len(file_paths_t))
captions = ["Ground Truth", "MindEye", "Brain Diffuser", "Tagaki et al."]
# make_results_figure(f"output/mental_imagery_paper/figures/mi_{mode}_simple_low_level.png", file_paths_t, captions, rows=6, columns=4)
images = []
for file in file_paths:
    images.append(Image.open(file))
if mode == "vision":
    title =  f"Low level reconstructions for seen {stim} stimuli"
else:
    title =  f"Low level reconstructions for imagined {stim} stimuli"
tiled_figure = format_tiled_figure(
    images=images,
    captions=captions, 
    rows = 4, 
    cols = 6,
    red_line_index=0,  # Replace with the index of the column/row to highlight red
    buffer=20, # The buffer space in pixels between images.
    mode=3, # The mode of how captions and titles are used.
    title = title
    )
tiled_figure.save(f"output/mental_imagery_paper/image_grids/mi_{mode}_{stim}_low_level.png")

0 0
6 6
6 6
6 6
24


/tmp/ipykernel_73404/3692248890.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Rank_Sum'] = ranked_df.sum(axis=1)
/tmp/ipykernel_73404/3692248890.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Rank_Sum'] = ranked_df.sum(axis=1)
/tmp/ipykernel_73404/3692248890.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat

In [3]:
from IPython.display import display
def create_word_images(word_list):
    images = []

    for word in word_list:
        # Create an image with white background
        image = Image.new('RGB', (768, 768), color='white')
        draw = ImageDraw.Draw(image)

        # Set a fixed, reasonable font size
        font_size = 500
        font = ImageFont.load_default(size=font_size)

        # Calculate the position for the text to be centered
        text_size = draw.textsize(word, font=font)
        text_x = (image.width - text_size[0]) / 2
        text_y = (image.height - text_size[1]) / 2

        # Draw the text in black
        draw.text((text_x, text_y), word, fill='black', font=font)

        images.append(image)

    return images
image = create_word_images(["apple"])[0]
image.save("output/mental_imagery_paper/image_grids/apple.png")

/tmp/ipykernel_110337/3620571825.py:15: DeprecationWarning: textsize is deprecated and will be removed in Pillow 10 (2023-07-01). Use textbbox or textlength instead.
  text_size = draw.textsize(word, font=font)


In [4]:

for mode in ["vision", "imagery"]:
    file_paths = []
    for method in ["mindeye", "brain-optimized-inference-v2.1", "braindiffuser", "brain-optimized-inference-v2.3", "mindeye2", "tagaki"]:
        directory_path = f"output/mental_imagery_paper/{mode}/{method}/subject1_statistics_18.csv"
        df = pd.read_csv(directory_path)
        for subject in [2,5,7]:
            new_df = pd.read_csv(f"output/mental_imagery_paper/{mode}/{method}/subject{subject}_statistics_18.csv")
            df = pd.concat([df, new_df])
        df.to_csv(f"output/mental_imagery_paper/{mode}/{method}/all_subject_statistics.csv")

        df.reset_index(drop=True, inplace=True)
        # Define the columns to be used for selecting the best row for each "ID"
        columns_to_consider = [
            "CLIP Cosine",
            "Brain Correlation V1",
            "Brain Correlation V2",
            "Brain Correlation V3",
            "Brain Correlation V4",
            "Brain Correlation Higher Visual",
            "Brain Correlation NSD General",
            "AlexNet 2",
            "AlexNet 5",
            "AlexNet 7",
            "CLIP Two-way",
            "Inception V3"
        ]

        filtered_df = df[(df['ID'] < 12) & (df['Sample Indicator'] == 12)]
            

        # Rank each of the relevant columns across the filtered DataFrame and sum these ranks for each row
        ranked_df = filtered_df[columns_to_consider].rank(ascending=False, method='min', na_option='keep')
        filtered_df['Rank_Sum'] = ranked_df.sum(axis=1)

        # Find the row with the lowest sum of ranks for each "ID"
        best_rows_indices_filtered = filtered_df.groupby('ID')['Rank_Sum'].idxmin()
        best_rows_filtered = filtered_df.loc[best_rows_indices_filtered].sort_values(by="ID")

        # Generate file paths for the images corresponding to the rows after filtering
        image_file_paths = [
            f"output/mental_imagery_paper/{row['Mode']}/{row['Method']}/subject{int(row['Subject'])}/{int(row['ID'])}/{int(row['Sample Count'])}.png"
            for index, row in best_rows_filtered.iterrows()
        ]

        # Generate file paths for the ground truth images corresponding to the filtered rows
        ground_truth_file_paths = [
            f"output/mental_imagery_paper/{row['Mode']}/{row['Method']}/subject{int(row['Subject'])}/{int(row['ID'])}/ground_truth.png"
            for index, row in best_rows_filtered.iterrows()
        ]
        # print(len(image_file_paths))
        if method == "mindeye":
            file_paths += ground_truth_file_paths
        file_paths += image_file_paths

    captions = ["Ground Truth", "MindEye1", "ME1 + BOI", "Brain Diffuser", "BD + BOI", "MindEye2", "Tagaki et al."]
    images = []
    for file in file_paths:
        images.append(Image.open(file))
    if mode == "vision":
        title =  f"Reconstructions for seen stimuli"
    else:
        title =  f"Reconstructions for imagined stimuli"
        
        
    tiled_figure = format_tiled_figure(
        images=images,
        captions=captions, 
        rows = 7, 
        cols = 12,
        red_line_index=0,  # Replace with the index of the column/row to highlight red
        buffer=10, # The buffer space in pixels between images.
        mode=3, # The mode of how captions and titles are used.
        title = title
        )
    tiled_figure.save(f"output/mental_imagery_paper/image_grids/mi_{mode}.png")
# tiled_figure.save(f"output/second_sight_paper/qualia_structure_figures/mi_{mode}_{stim}.png")

# make_results_figure(f"output/mental_imagery_paper/image_grids/mi_{mode}_{stim}.png", transpose_flat_list(file_paths, 5, 6), captions, rows=6, columns=5)
# make_results_figure(f"output/mental_imagery_paper/image_grids/mi_{mode}_{stim}_sideways.png", file_paths, captions, rows=5, columns=6)

/tmp/ipykernel_26098/1274092079.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Rank_Sum'] = ranked_df.sum(axis=1)
/tmp/ipykernel_26098/1274092079.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['Rank_Sum'] = ranked_df.sum(axis=1)
/tmp/ipykernel_26098/1274092079.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydat